<a href="https://colab.research.google.com/github/Location-Artistry/ColabNotebooks/blob/master/CustomArcGISPyAPIfunctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install and Import Libraries
**RUN FIRST**

In [ ]:
!pip install arcgis

In [4]:
from pathlib import Path
import sys
import os
import datetime as dt
import time as time
import pandas 
from arcgis.gis import GIS, Item
from arcgis.env import active_gis
from arcgis.features import FeatureLayerCollection
from arcgis.mapping import WebMap
from IPython.display import display
import getpass
from pathlib import Path
from zipfile import ZipFile

# USER LOGIN

In [ ]:
userLogin()

# Search Function

In [ ]:
searchInput = 'type:Web map'
searchByKeyViz(gis, searchInput)

# Update Feature Layer from Related Table with Python
Provide Feature Layer item.id and list of breakpoints for classification  
Will reclassify target attributes in Feature Layer based on most recent table record and breakpoint list  
If target attribute is display attributte for Webmap, display will be updated based on new value

In [ ]:
inputText = input(f'Item ID to update status: ')
print(f'Thank you, ID: {inputText}')

In [19]:
# this is an example updating the Dissolved Oxygen status for water sampling stations

targetLayerID = 'myWaterQualityLayerItemID'
match = 'SITE_ID' # this is the identifier between the features and table
DOtarget = 'DO_STATUS_' # this is the target attributte to update
DOsource = 'DO_mgl' # this is the source atrribute of the update data
DOrefList = [5,6,7,100]

# reference function definition
# updateLayFeatFromTableBreaks (gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList)
# will update the water stations with most recent Dissolved Oxygen Status: [1=POOR,2=MODERATE,3=GOOD,4=EXCELLENT]
updateLayFeatFromTableBreaks(gis, targetLayerID, match, DOtarget, DOsource, DOrefList)

none
none
feature: ICD-ST-40 from tableFeature: ICD-ST-40 set DO_STATUS_: 6.57 as: 3
feature: PNC-ST-45 from tableFeature: PNC-ST-45 set DO_STATUS_: 0.99 as: 1
feature: PNC-ST-45 from tableFeature: PNC-ST-45 set DO_STATUS_: 0.96 as: 1
feature: PNC-ST-50 from tableFeature: PNC-ST-50 set DO_STATUS_: 1.04 as: 1
feature: PNC-ST-60 from tableFeature: PNC-ST-60 set DO_STATUS_: 2.14 as: 1
none
none
none
feature: SCD-ST-40 from tableFeature: SCD-ST-40 set DO_STATUS_: 7.67 as: 4
feature: NOT-ST-120 from tableFeature: NOT-ST-120 set DO_STATUS_: 4.04 as: 1
feature: NOT-ST-120 from tableFeature: NOT-ST-120 set DO_STATUS_: 4.26 as: 1
feature: NOT-ST-110 from tableFeature: NOT-ST-110 set DO_STATUS_: 4.34 as: 1
feature: NOT-ST-70 from tableFeature: NOT-ST-70 set DO_STATUS_: 4.83 as: 1
none
feature: NOT-ST-70 from tableFeature: NOT-ST-70 set DO_STATUS_: 5.49 as: 2
feature: PNC-ST-40 from tableFeature: PNC-ST-40 set DO_STATUS_: 1.54 as: 1
feature: PNC-ST-40 from tableFeature: PNC-ST-40 set DO_STATUS_: 

# Python Functions from my ArcGIS Python Colab Notebook  
**MUST RUN THIS CELL AFTER INSTALL AND IMPORT THEN - userLogin() -**


In [18]:
# Collection of all ArcGIS Python API Helper Functions
# user login functions, ask if user would like additional logins
def userLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis
        gis = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis.users.me.username} ACCOUNT as <gis>')
        print(gis)
        addUsers = input(f'Additional User Login(YES/NO)? ')
        if addUsers.upper() == 'YES':
            additionalUserLogin()
        else:
            print(f'YOU MAY NOW PROCEED...')
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

def additionalUserLogin():
    userID = input(f'ArcGIS Online USER ID: ')
    passWord = getpass.getpass('PASSWORD: ')
    try:
        global gis2
        gis2 = GIS("https://www.arcgis.com", userID, passWord)
        print(f'SUCCESS - CONNECTED TO: {gis2.users.me.username} ACCOUNT as <gis2>')
        print(gis2)
    except:
        print(f'ERROR DID NOT CONNECT TO: {userID}')

# get list of all owner AGOL items, print list with title, id, type, and categories
def getUserContent(gisInfo):
    try:
        my_content = gisInfo.content.search(query="owner:" + gisInfo.users.me.username, item_type="", max_items=200)
        for x in my_content:
            print(f'{x.title} - {x.id} - {x.type} - {x.categories}')          
            # print(x.title +' - ' + x.id + ' - ' + x.type + ' -',x.categories)
    except:
        print('ERROR could not get user content')

# Clone item using id of item passed to function
def cloneItem(gisInfo, gisInfo2, cloneID):    
    try:
        itemToClone = gisInfo.content.get(cloneID)
        print('Cloning:' + itemToClone.title + ' - ' + itemToClone.id + ' -',itemToClone.type)
        clonedItem = gisInfo2.content.clone_items(items=[itemToClone])
        print(f'Cloned Item: {clonedItem[0]}')
        #return clonedItem
    except:
        print('ERROR Could Not Clone')

# find item by keywords and display text information
def searchByKeywords(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        for z in searchContent:
            print(f'title: {z.title} - itemID: {z.id} - type: {z.type} - dateEdited: ')
    except:
        print('ERROR Search not Successful')

# find item by keywords and display visual card
def searchByKeyViz(gisInfo, searchKeywords):
    try:
        searchContent = gisInfo.content.search(query=f'{searchKeywords}', item_type='', max_items=50)
        for z in searchContent:
            print(f'title: {z.title} - itemID: {z.id} - type: {z.type}')
            display(z)
    except:
        print('ERROR Search not Successful')
        
# return all keys and values for item when passed itemID string
def getItemKeysValues(gisInfo, idString):
    try:
        getFeature = gisInfo.content.get(idString)
        for key, value in getFeature.items():
            print(key,': ', value)
    except:
        print('ERROR GET Keys/Values not Successful')
        
# takes itemID and gets and returns layerObject if exist, otherwise 'no layers found'
def getLayers(gisInfo, idString):
    getFeature = gisInfo.content.get(idString)
    try:
        featureLayers = getFeature.layers
        z = 0 
        for x in featureLayers:
            print(f'Layer {z}: {x}')
            z += 1
    except:
        print('no layers found')
    return featureLayers

# supply feature layer itemID, and the layer number to display table head 
def getLayerTable(gisInfo, idString, layerNum):
    try:
        layerOutput = getLayers(gisInfo, idString)
        queryLayer = layerOutput[layerNum].query()
        display(queryLayer.sdf.head())
    except:
        print('ERROR no Layers Found')
        
# delete item by itemID
def deleteItem(gisInfo, idString):
    itemToDelete = gisInfo.content.get(idString)
    display(itemToDelete)
    delQuest = input(f'Are you sure you want to delete: {itemToDelete.title}')
    try:
        if delQuest.upper() == 'YES':
            print(f'DELETING: {itemToDelete.title}')
            itemToDelete.delete()
        else:
            print(f'NOT DELETING: {itemToDelete.title}')
    except:
        print(f'ERROR failed to DELETE: {itemToDelete.title}')

# List all user Dashboards and Dashboard Webmmaps
def ListAllDashWebmaps(gisInfo):
  source_admin_inventory = get_user_items(gisInfo, gisInfo.users.me)
  x = 0
  try:
    for dashboard in source_admin_inventory['Dashboard']:
        print(x, dashboard)
        dashWebmap = get_dash_wm(gisInfo, dashboard)
        print(dashWebmap)
        x += 1
  except:
    print("ERROR COULD NOT LIST DASHBOARDS") 

# generic function update targetLayer Features based on Table Records
def updateLayFeatFromTable(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      tableFeatureID = tableFeature.attributes[matchAttrib]
      for layerFeature in layerFeatures:
        layerFeatureID = layerFeature.attributes[matchAttrib]
        if tableFeatureID == layerFeatureID:
          targetValue = tableFeature.attributes[sourceAttrib]
          layerFeature.set_value(targetAttrib, targetValue)
          print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib} as: {targetValue}')
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
  except:
    print(f'update features failed for {getLayers.title}')
  
# generic function update targetLayer Features based on Table Record, adds break list for parameter categories mapping/analysis
def updateLayFeatFromTableBreaks(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList):
  try:
    getLayers = gisInfo.content.get(targetLayerID)
    targetLayer = getLayers.layers
    layerFeatures = targetLayer[0].query()
    sourceTable = getLayers.tables
    tableFeatures = sourceTable[0].query()
    for tableFeature in tableFeatures:
      tableFeatureID = tableFeature.attributes[matchAttrib]
      for layerFeature in layerFeatures:
        layerFeatureID = layerFeature.attributes[matchAttrib]
        if tableFeatureID == layerFeatureID:
          targetValue = tableFeature.attributes[sourceAttrib]
          x = 1
          for breakVal in breaksList:
            if targetValue > breakVal:
              print('none')
            else:
              layerFeature.set_value(targetAttrib, x)
              print(f'feature: {layerFeatureID} from tableFeature: {tableFeatureID} set {targetAttrib}: {targetValue} as: {x}')
              break
            x+=1
    layerEdits = targetLayer[0].edit_features(updates=layerFeatures)
    editCounter = 0
    for edits in layerEdits['updateResults']:
      editCounter+=1
    # hide REST infor for updated layers and tables
    # print(f'updated {getLayers.title} with {editCounter} edits from {getLayers.tables[0]}')
    print(f'updated {getLayers.title} with {editCounter} edits')
  except:
    print(f'update features failed for {getLayers.title}')

# download Feature Layer data from AGOL, unzip contents to folder with item.title name
# Export Formats: Shapefile | CSV | File Geodatabase | Feature Collection | GeoJson | Scene Package | KML | Excel
def downloadItem(gisInfo, idString):
    try:
        downloadData = gisInfo.content.get(idString)
        dataPath = Path('/data')
        print(f'Downloading: {downloadData.title} to {dataPath} directory')
        if not dataPath.exists():
          dataPath.mkdir()
        # this portion for feature service
        downloadExport = downloadData.export(title=downloadData.title, export_format="CSV")
        zipPath = downloadExport.download(save_path=dataPath)
        # preparing to extract files to directory with item.title name
        #zipPath = downloadData.download(save_path=dataPath)
        extractPath = dataPath.joinpath(downloadData.title)
        # extract files to /data directory
        zipFiles = ZipFile(zipPath)
        zipFiles.extractall(path=extractPath)
        print(f'list of Files extracted to: {extractPath}')
        print(list(file.name for file in extractPath.glob('*')))
    except:
        print('ERROR DOWNLOAD did not workings!')
    
# *******ESRI pre-made helper functions********
def is_hosted(gisInfo, item):
    return [keyword for keyword in item.typeKeywords if "Hosted" in keyword] 

# Prints all layers in a webmap, very handy
def print_webmap_inventory(gisInfo, wm):
    wm_obj = WebMap(wm)
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        try:
            if is_hosted(Item(gisInfo, wm_layer['itemId'])):
                print(f"{' '*2}{wm_layer['title']:40}HOSTED{' ':5}"
                      f"{wm_layer['layerType']:20}{dict(wm_layer)['itemId']}")
            else:
                print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                      f"{wm_layer['layerType']:20}{wm_layer.id}") 
        except:
            print(f"{' '*2}{wm_layer['title']:40}other{' ':6}"
                  f"{wm_layer['layerType']:20}{wm_layer.id}")
    print("\n")

def get_webmap_list(wm):
    wm_obj = WebMap(wm)
    wmList = []
    print(f"{wm_obj.item.title}\n{'-'*100}")
    for wm_layer in wm_obj.layers:
        # print(wm_layer.itemId)
        wmList.append(wm_layer.itemId)
    return(wmList)
    
def displayWebmapLayers(gisInfo, idList):
    for id in idList:
        displayLayer = gisInfo.content.get(id)
        display(displayLayer)

def get_user_items(gisInfo, user):
    user_inventory = {}
    user_items = gisInfo.content.search(query=f"* AND owner:{user.username}", 
                                           max_items=500)
    for item in user_items:
        if item.type not in user_inventory:
            user_inventory[item.type] = [i 
                                         for i in user_items 
                                         if i.type == item.type]
    return user_inventory

def print_user_inventory(inventory):
    for itype, ilist in inventory.items():
        try:
            print(f"{itype}\n{'-'*50}")
            for i in ilist:
                print(f"{' ':3}{i.title:50}")
            print("\n")
        except Exception as e:
            print(f"\t\tOperation failed on: {i.title}")
            print(f"\t\tException: {sys.exc_info()[1]}")
            continue
            
def get_dash_wm(gisInfo, dash):
    return [gisInfo.content.get(widget['itemId']) 
            for widget in dash.get_data()['widgets'] 
            if widget['type'] == "mapWidget"]

# Python API Function List  
**userLogin()** - **additionalUserLogin()** - **getUserContent** *(gisInfo)* - **cloneItem** *(gisInfo, gisInfo2, cloneID)*  
**searchByKeywords** *(gisInfo, searchKeywords)* - **searchByKeyViz** *(gisInfo, searchKeywords)*  
**getItemKeysValues** *(gisInfo, idString)* - **getLayers** *(gisInfo, idString)*  
**getLayerTable** *(gisInfo, idString, layerNum)* - **deleteItem** *(gisInfo, idString)*  
**ListAllDashWebmaps** *(gisInfo)* - **updateLayFeatFromTable** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib)*  
**updateLayFeatFromTableBreaks** *(gisInfo, targetLayerID, matchAttrib, targetAttrib, sourceAttrib, breaksList)*  
**downloadItem** *(gisInfo, idString)* 


---


# ESRI pre-made helper functions  
**is_hosted** *(gisInfo, item)* - **print_webmap_inventory** *(gisInfo, wm)* - **get_webmap_list** *(wm)*  
**displayWebmapLayers** *(gisInfo, idList)* - **get_user_items** *(gisInfo, user)*  
**print_user_inventory** *(inventory)* - **get_dash_wm** *(gisInfo, dash)*

